In [83]:
import requests 
import json
top_stories_response = requests.get('https://hacker-news.firebaseio.com/v0/topstories.json?print=pretty')
top_stories_ids = json.loads(top_stories_response.content)

In [91]:
news = []
urls = []
s = requests.Session()
for idk in top_stories_ids:
    url = 'https://hacker-news.firebaseio.com/v0/item/{}.json?print=pretty'.format(idk)
    urls.append(url)
    response = s.get(url).json()
    news.append(response['title'])



In [85]:
news[:10]

['The Wrong Abstraction (2016)',
 'Netflix Makes a Statement in India with “Sacred Games”',
 'College Rejection Threat Highlights Social Credit Blacklists',
 'Public.resource.org wins appeal on right to publish the law [pdf]',
 'Doing Windows, Part 5: A Second Try',
 'Riving, a Viking-age woodworking technique',
 'Why Philosophers Should Care About Computational Complexity [pdf]',
 'The Octonion Math That Could Underpin Physics',
 'PeerTube, the “Decentralized YouTube”, succeeds in crowdfunding',
 'DNS Zombies (2016)']

In [86]:
import gensim
news_prep = [gensim.parsing.preprocessing.preprocess_string(n) for n in news]

news_prep[:3]

[['wrong', 'abstract'],
 ['netflix', 'make', 'statement', 'india', '“sacr', 'games”'],
 ['colleg', 'reject', 'threat', 'highlight', 'social', 'credit', 'blacklist']]

In [87]:
word2vec = gensim.models.Word2Vec(news_prep, size=100,window=10, min_count=2, workers=10)

In [88]:
word2vec.train(news_prep, total_examples=len(news_prep), epochs=10)

(8839, 20460)

In [90]:
word2vec.wv.most_similar('social')

[('passport', 0.3877919912338257),
 ('plai', 0.3261300027370453),
 ('practic', 0.29943904280662537),
 ('moon', 0.2781459391117096),
 ('game', 0.2151266187429428),
 ('netflix', 0.19887347519397736),
 ('warn', 0.19503644108772278),
 ('find', 0.1925685852766037),
 ('control', 0.186159148812294),
 ('app', 0.17776241898536682)]

In [96]:
html = s.get(s.get(urls[0]).json()['url']).text

In [98]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(html, 'lxml')

In [102]:
ps = soup.findAll('p')
len(ps)

35

In [115]:
content = ""
for p in ps:
    content += p.getText() + '.'

In [117]:
content = gensim.parsing.preprocessing.strip_multiple_whitespaces(content)
content

'_I originally wrote the following for my Chainline Newsletter, but I continue to get tweets about this idea, so I\'m re-publishing the article here on my blog. This version has been lightly edited._.I\'ve been thinking about the consequences of the "wrong abstraction." My RailsConf 2014 "all the little things" talk included a section where I asserted:.duplication is far cheaper than the wrong abstraction.And in the summary, I went on to advise:.prefer duplication over the wrong abstraction.This small section of a much bigger talk invoked a surprisingly strong reaction. A few folks suggested that I had lost my mind, but many more expressed sentiments along the lines of:..This, a million times this! "@BonzoESC: "Duplication is far cheaper than the wrong abstraction" @sandimetz @rbonales pic.twitter.com/3qMI0waqWb"..The strength of the reaction made me realize just how widespread and intractable the "wrong abstraction" problem is. I started asking questions and came to see the following 

In [122]:
sum_c = gensim.summarization.summarize(content, split=True)
sum_c

['Programmer A trots happily away..Time passes..A new requirement appears for which the current abstraction is almost perfect..Programmer B gets tasked to implement this requirement.',
 ". Programmer B feels honor-bound to retain the existing abstraction, but since isn't exactly the same for every case, they alter the code to take a parameter, and then add logic to conditionally do the right thing based on the value of that parameter..",
 'What was once a universal abstraction now behaves differently for different cases..Another new requirement arrives.Programmer X.Another additional parameter.Another new conditional.Loop until code becomes incomprehensible..You appear in the story about here, and your life takes a dramatic turn for the worse..Existing code exerts a powerful influence.',
 'And, unfortunately, the sad truth is that the more complicated and incomprehensible the code, i.e. the deeper the investment in creating it, the more we feel pressure to retain it (the "sunk cost fal